In [33]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [105]:
outages_df = pd.read_excel('data/original/Book2(1).xlsx', usecols=['Унікод АЗС', 'ДатаЧас', 'Обсяг спожитих, кВт', 'Year', 'Month', 'Date', 'Day', 'DayWeek', 'Hour'])

In [106]:
latitude = []
longitude = []

for i in outages_df['Унікод АЗС']:
    if i == 40810600:
        latitude.append(50.418366003690984)
        longitude.append(29.840546770260875)
    if i == 40861000:
        latitude.append(50.31828636812571)
        longitude.append(31.03586158987561)
    if i == 40860500:
        latitude.append(50.319652674621935)
        longitude.append(31.03398680305577)
    if i == 40810700:
        latitude.append(50.42274230627166)
        longitude.append(29.843808164165303)

outages_df['Latitude'] = latitude
outages_df['Longitude'] = longitude


In [107]:
outages_df

,Унікод АЗС,ДатаЧас,"Обсяг спожитих, кВт",Year,Month,Date,Day,DayWeek,Hour,Latitude,Longitude
0,40810600,2023-06-01 00:00:00,29.3216,2023,6,2023-06-01,1,3,0,50.418366,29.840547
1,40810600,2023-06-01 01:00:00,28.2432,2023,6,2023-06-01,1,3,1,50.418366,29.840547
2,40810600,2023-06-01 02:00:00,25.5184,2023,6,2023-06-01,1,3,2,50.418366,29.840547
3,40810600,2023-06-01 03:00:00,28.1296,2023,6,2023-06-01,1,3,3,50.418366,29.840547
4,40810600,2023-06-01 04:00:00,26.8352,2023,6,2023-06-01,1,3,4,50.418366,29.840547
...,...,...,...,...,...,...,...,...,...,...,...
49921,40810700,2024-11-01 19:00:00,59.8150,2024,11,2024-11-01,1,4,19,50.422742,29.843808
49922,40810700,2024-11-01 20:00:00,69.7740,2024,11,2024-11-01,1,4,20,50.422742,29.843808
49923,40810700,2024-11-01 21:00:00,75.8230,2024,11,2024-11-01,1,4,21,50.422742,29.843808
49924,40810700,2024-11-01 22:00:00,70.4940,2024,11,2024-11-01,1,4,22,50.422742,29.843808


In [108]:
outages_list = []
kW_list = []
for i in outages_df['Обсяг спожитих, кВт']:
    if pd.isna(i):
        outages_list.append(False)
        kW_list.append(0)
    else:
        outages_list.append(True)
        kW_list.append(i)

outages_df['Light On'] = outages_list
outages_df['kW'] = kW_list

In [109]:
outages_df = outages_df.drop(columns = ['Date'])
# outages_df = outages_df.drop(columns = ['ДатаЧас'])
outages_df = outages_df.drop(columns = ['Обсяг спожитих, кВт'])

In [113]:
X = outages_df.drop(columns=['kW'])
y = outages_df[['kW', 'Year', 'Month']] 

In [112]:
X_val = X[(X['Year'] ==2024) & (X['Month'] == 10)]
y_val = y[(y['Year'] ==2024) & (y['Month'] == 10)]

X_test = X[(X['Year'] ==2024) & (X['Month'] == 11)]
y_test = y[(y['Year'] ==2024) & (y['Month'] == 11)]

X_train = X[~((X['Year'] == 2024) & (X['Month'] == 11))]
y_train = y[~((y['Year'] == 2024) & (y['Month'] == 11))]

X_train = X_train[~((X_train['Year'] == 2024) & (X_train['Month'] == 10))]
y_train = y_train[~((y_train['Year'] == 2024) & (y_train['Month'] == 11))]

       Унікод АЗС             ДатаЧас  Year  Month  Day  DayWeek  Hour  \
11713    40810600 2024-10-01 00:00:00  2024     10    1        1     0   
11714    40810600 2024-10-01 01:00:00  2024     10    1        1     1   
11715    40810600 2024-10-01 02:00:00  2024     10    1        1     2   
11716    40810600 2024-10-01 03:00:00  2024     10    1        1     3   
11717    40810600 2024-10-01 04:00:00  2024     10    1        1     4   
...           ...                 ...   ...    ...  ...      ...   ...   
49897    40810700 2024-10-31 19:00:00  2024     10   31        3    19   
49898    40810700 2024-10-31 20:00:00  2024     10   31        3    20   
49899    40810700 2024-10-31 21:00:00  2024     10   31        3    21   
49900    40810700 2024-10-31 22:00:00  2024     10   31        3    22   
49901    40810700 2024-10-31 23:00:00  2024     10   31        3    23   

        Latitude  Longitude  Light On  
11713  50.418366  29.840547      True  
11714  50.418366  29.840547    

In [69]:
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [70]:
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
np.sqrt(mse)

19.91262876109946

In [73]:
import xgboost as xgb

In [75]:
max_depth_list = [4, 6, 8, 10, 12, 16, 18, 20]
n_estimators_list = [50, 100, 200, 500, 1000]
learning_rate_list = [0.1, 0.05, 0.01]
best_max_depth = None
best_n_estimators = None
best_learning_rate = None
best_validation_preds = None
best_rmse = float('inf')
best_xgb_model = None

for max_depth in max_depth_list:
    for n_estimators in n_estimators_list:
        for learning_rate in learning_rate_list:
            xgb_model = xgb.XGBRegressor(n_estimators=n_estimators, max_depth=max_depth, learning_rate=learning_rate, n_jobs=4, tree_method='auto')
            xgb_model.fit(X_train, y_train)
            validation_preds = xgb_model.predict(X_val)
            validation_preds = pd.Series(validation_preds)
            validation_target = pd.Series(y_val.values)
            err = np.sqrt(mse(validation_target, validation_preds))
            if err < best_rmse:
                best_rmse = err
                best_max_depth = max_depth
                best_n_estimators = n_estimators
                best_learning_rate = learning_rate
                best_validation_preds = validation_preds
                best_xgb_model = xgb_model

print(f'XGB RMSE: {best_rmse}')
print(f'max_depth: {best_max_depth}, n_estimators: {best_n_estimators}, learning_rate: {best_learning_rate}')

TypeError: 'numpy.float64' object is not callable